# OpenRouteService python bindings


TESTING ORS v6.2 VROOM v1.6.0 ORS-PY v2.3.0

In [5]:
import openrouteservice as ors
import folium
import json
import random
from pprint import pprint

url_ors = 'http://ors:8080/ors'  #istanze locale ROUTING ENGINE 

url_vroom = 'http://vroom:8081'  #instanza loale VROOM

# DATI json posizione jobs (Bidoni)
pointsA = json.load(open('./data/mezzolombardo.geojson'))['features'] #4 points
pointsB = json.load(open('./data/mezzocorona.geojson'))['features'] #6 points

#shipments = json.load(open('./data/shipments.json'))

center = [46.2147, 11.1065]
zoom = 15

# POSIZIONI PARTENZA DEI VEICOLI (Furgoni)
point_start = [46.2170,11.1066]

# POSIZIONE ARRIVO DEI VEICOLI
point_end = [point_start[0], point_start[1]]

ccA = [p['geometry']['coordinates'] for p in pointsA]
ccB = [p['geometry']['coordinates'] for p in pointsB]
#ELENCO BIDONI 4 Mezzolombardo e 6 Mezzocorona TOTALE 10 con CONTENUTO CASUALE
job_locs = ccA + ccB

In [6]:
ors.__version__

'2.3.0'

In [11]:
map1 = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)

vehicles = []

#vehicle capacity
capacity = 5

vehicles.append(ors.optimization.Vehicle(
    id = 1,
    capacity = [capacity],  # Limite di capacita' del Veicolo    
    profile = 'driving-car',
    start = list(reversed(point_start)), # posizione di partenza veicoli
    end = list(reversed(point_start)),
))
mark = folium.Marker(point_start, tooltip=folium.Tooltip(capacityB,permanent=True), icon=folium.Icon(icon='truck', prefix='fa', color='blue'))
mark.add_to(map1)

jobs = []
for idx, cc in enumerate(job_locs):
    loc = list(reversed(cc))
    
    amount = random.randrange(1, 3) #random value of bin filling
    
    mark = folium.Marker(loc, tooltip=folium.Tooltip(pickup, permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green'))
    mark.add_to(map1)

    jobs.append(ors.optimization.Job(
        id = idx,
        location = cc,
        pickup = [pickup], # Quantita' contenuta nel Bidone
        #priority = [1-100] potrebbe essere utile anche priority
        #ma non e' supportato, bug che verra presto risolto https://github.com/GIScience/openrouteservice-py/issues/39
    ))

map1

TypeError: __init__() got an unexpected keyword argument 'pickup'

In [24]:
vehicles
jobs

[]

In [23]:
clientVrp = ors.Client(base_url=url_vroom, key='')

# USIAMO VROOM ATTRAVERSO LA API PYTHON DI ORS

solution = clientVrp.optimization(
    vehicles = vehicles,
    jobs = jobs,
    geometry = True   #VROOM RESTITUISCE LA GEOMETRIA DEL PERCORSO OLTRE ALLA SOLUZIONE
)

#SOLUZIONE CALCOLATA DA VROOM

solutionA = solution['routes'][0]

poly = folium.PolyLine(
    locations = [list(reversed(coords)) for coords in ors.convert.decode_polyline(solutionA['geometry'])['coordinates']],
    color = 'blue',
    tooltip = folium.Tooltip('Distance: '+str(solutionA['distance']/1000)+'km', permanent=True)
)

poly.add_to(map1)

map1

ApiError: 400 ({'code': 2, 'error': 'Invalid jobs or shipments.'})

In [17]:
solution

{'code': 0,
 'summary': {'cost': 269,
  'unassigned': 2,
  'delivery': [5],
  'amount': [5],
  'pickup': [0],
  'service': 0,
  'duration': 269,
  'waiting_time': 0,
  'distance': 3340,
  'computing_times': {'loading': 15, 'solving': 1, 'routing': 11}},
 'unassigned': [{'id': 7,
   'location': [11.127262115478516, 46.211972058400264]},
  {'id': 3, 'location': [11.096470355987549, 46.21344204466888]}],
 'routes': [{'vehicle': 1,
   'cost': 269,
   'delivery': [5],
   'amount': [5],
   'pickup': [0],
   'service': 0,
   'duration': 269,
   'waiting_time': 0,
   'distance': 3340,
   'steps': [{'type': 'start',
     'location': [11.1066, 46.217],
     'load': [5],
     'arrival': 0,
     'duration': 0,
     'distance': 0},
    {'type': 'job',
     'location': [11.123356819152832, 46.21419929496722],
     'job': 6,
     'service': 0,
     'waiting_time': 0,
     'load': [3],
     'arrival': 107,
     'duration': 107,
     'distance': 1444},
    {'type': 'job',
     'location': [11.118206977